# Evaporation and cover measurment initials

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pvlib
!pip install timezonefinder
!pip install numpy-financial
!pip install nrel-pysam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 33.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.3 MB/s eta 0:00:00
  Created wheel for timezonefinder: filename=timezonefinder-6.2.0-cp310-cp310-manylinux_2_35_x86_64.whl size=46900638 sha256=7a453abba2f38786c0daafb4bf75366674a33ce49234e7a2c40ab88be088ce5f
  Stored in directory: /root/.cache/pip/wheels/17/19/d4/ae94459b7f74f7e8f171862d1c08adedf9b7c76ddfc514a620
Successfully built timezonefinder
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 18.0 MB/s eta 0:00:00


# For any location evaporation measurement using NREL API KEy

In [ ]:
import io
import requests
import pandas as pd
import pvlib
from json import JSONDecodeError
import warnings
from pvlib._deprecation import pvlibDeprecationWarning
import numpy as np
import numpy_financial as npf
import datetime
import xarray as xr
import math
#import cupy as cp
from timezonefinder import TimezoneFinder
from pvlib.tools import sind
from pvlib._deprecation import warn_deprecated
from pvlib.tools import _get_sample_intervals
import scipy
import scipy.constants
import warnings
import json
import PySAM.Pvsamv1 as pvsam
from PySAM import Utilityrate5 as ur5  # Add this import statement
#import pysam
import os
from io import StringIO
import copy
import tempfile
from math import pi, cos, sin, tan, acos, sqrt, log, exp




# 'relative_humidity', 'total_precipitable_water' are not available
ATTRIBUTES = (
    'air_temperature', 'dew_point', 'dhi', 'dni', 'ghi', 'surface_albedo',
    'surface_pressure', 'wind_direction', 'wind_speed')
PVLIB_PYTHON = 'pvlib python'

# Dictionary mapping PSM3 names to pvlib names
VARIABLE_MAP = {
    'GHI': 'ghi',
    'DHI': 'dhi',
    'DNI': 'dni',
    'Clearsky GHI': 'ghi_clear',
    'Clearsky DHI': 'dhi_clear',
    'Clearsky DNI': 'dni_clear',
    'Solar Zenith Angle': 'solar_zenith',
    'Temperature': 'temp_air',
    'Relative Humidity': 'relative_humidity',
    'Dew point': 'temp_dew',
    'Pressure': 'pressure',
    'Wind Direction': 'wind_direction',
    'Wind Speed': 'wind_speed',
    'Surface Albedo': 'albedo',
    'Precipitable Water': 'precipitable_water',
}


def get_psm3(latitude, longitude, api_key, email, names='tmy', interval=60,
             attributes=ATTRIBUTES, leap_day=None, full_name=PVLIB_PYTHON,
             affiliation=PVLIB_PYTHON, map_variables=None, timeout=30, reservoir_name=""):
    # The well know text (WKT) representation of geometry notation is strict.
    # A POINT object is a string with longitude first, then the latitude, with
    # four decimals each, and exactly one space between them.
    longitude = ('%9.4f' % longitude).strip()
    latitude = ('%8.4f' % latitude).strip()
    # TODO: make format_WKT(object_type, *args) in tools.py

    # convert to string to accomodate integer years being passed in
    names = str(names)

    # convert pvlib names in attributes to psm3 convention (reverse mapping)
    # unlike psm3 columns, attributes are lower case and with underscores
    amap = {value: key.lower().replace(' ', '_') for (key, value) in
            VARIABLE_MAP.items()}
    attributes = [amap.get(a, a) for a in attributes]
    attributes = list(set(attributes))  # remove duplicate values

    if (leap_day is None) and (not names.startswith('t')):
        warnings.warn(
            'The ``get_psm3`` function will default to leap_day=True '
            'starting in pvlib 0.11.0. Specify leap_day=True '
            'to enable this behavior now, or specify leap_day=False '
            'to hide this warning.', pvlibDeprecationWarning)
        leap_day = False

    # required query-string parameters for request to PSM3 API
    params = {
        'api_key': api_key,
        'full_name': full_name,
        'email': email,
        'affiliation': affiliation,
        'reason': PVLIB_PYTHON,
        'mailing_list': 'false',
        'wkt': 'POINT(%s %s)' % (longitude, latitude),
        'names': names,
        'attributes':  ','.join(attributes),
        'leap_day': str(leap_day).lower(),
        'utc': 'false',
        'interval': interval
    }
    NSRDB_API_BASE = "https://developer.nrel.gov"
    PSM_URL1 = NSRDB_API_BASE + "/api/nsrdb/v2/solar/psm3-download.csv"
    TMY_URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/psm3-tmy-download.csv"
    PSM5MIN_URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/psm3-5min-download.csv"

    # First, check longitude conditions
    if -16 < lon < 91:
        PSM_URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/msg-iodc-download.csv"
    elif 91 < lon < 182:
        PSM_URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/himawari-download.csv"
    else:
        # Then check latitude conditions if longitude didn't match
        if lat >= -21.186575058950382:
            PSM_URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/psm3-2-2-tmy-download.csv"
        else:
            PSM_URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/full-disc-download.csv"

    # Finally, set URL based on interval and 'names' prefix
    if any(prefix in names for prefix in ('tmy', 'tgy', 'tdy')):
        URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/psm3-2-2-tmy-download.csv"
    elif interval in (5, 15):
        URL = PSM5MIN_URL
    else:
        URL = PSM_URL

    response = requests.get(URL, params=params, timeout=timeout)

    if not response.ok:
        # if the API key is rejected, then the response status will be 403
        # Forbidden, and then the error is in the content and there is no JSON
        try:
            errors = response.json()['errors']
        except JSONDecodeError:
            errors = response.content.decode('utf-8')
        raise requests.HTTPError(errors, response=response)
    # the CSV is in the response content as a UTF-8 bytestring
    # to use pandas we need to create a file buffer from the response
    fbuf = io.StringIO(response.content.decode('utf-8'))
    return parse_psm3(fbuf, map_variables, reservoir_name)



def parse_psm3(fbuf, map_variables=None, reservoir_name=""):
    # Read metadata
    metadata_fields = fbuf.readline().split(',')
    metadata_fields[-1] = metadata_fields[-1].strip()
    metadata_values = fbuf.readline().split(',')
    metadata_values[-1] = metadata_values[-1].strip()
    metadata = dict(zip(metadata_fields, metadata_values))

    # Set some metadata types to numbers
    metadata['Local Time Zone'] = int(metadata['Local Time Zone'])
    metadata['Time Zone'] = int(metadata['Time Zone'])
    metadata['Latitude'] = float(metadata['Latitude'])
    metadata['Longitude'] = float(metadata['Longitude'])
    metadata['Elevation'] = int(metadata['Elevation'])

    # Read weather data
    columns = fbuf.readline().split(',')
    columns[-1] = columns[-1].strip()
    columns = [col for col in columns if col != '']
    dtypes = dict.fromkeys(columns, float)
    dtypes.update(Year=int, Month=int, Day=int, Hour=int, Minute=int)
    dtypes['Cloud Type'] = int
    dtypes['Fill Flag'] = int
    data = pd.read_csv(
        fbuf, header=None, names=columns, usecols=columns, dtype=dtypes,
        delimiter=',', lineterminator='\n')

    # Convert date vector to datetime
    dtidx = pd.to_datetime(
        data[['Year', 'Month', 'Day', 'Hour', 'Minute']])
    tz = 'Etc/GMT%+d' % -metadata['Time Zone']
    data.index = pd.DatetimeIndex(dtidx).tz_localize(tz)

    # Rename PSM3 variable names to SAM-compatible names
    data = data.rename(columns=VARIABLE_MAP)
    # Rearrange columns to match the expected order
    #data = data[['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'GHI', 'Dew Point', 'Temperature', 'Pressure', 'Wind Direction', 'Wind Speed', 'Surface Albedo']]
    data=data[['Year', 'Month', 'Day', 'Hour', 'Minute', 'dni', 'dhi', 'ghi', 'Dew Point', 'temp_air', 'pressure', 'wind_direction', 'wind_speed', 'albedo']]
    #Year	Month	Day	Hour	Minute	wind_speed	dhi	ghi	pressure	wind_direction	albedo	dni	temp_air	Dew Point

    # Add a "Timestamp" column
    #data['Timestamp'] = data.index.strftime('%Y%m%d:%H%M')

    # Create SAM-compatible file header
    header = [
        'Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,Dew Point Units,DHI Units,DNI Units,GHI Units,Temperature Units,Pressure Units,Wind Direction Units,Wind Speed Units,Surface Albedo Units,Version\n',
        'NSRDB,{},-,-,-,{},{},{},{},{},C,w/m2,w/m2,w/m2,C,mbar,Degrees,m/s,N/A,3.1.0\n'.format(metadata["Location ID"], metadata["Latitude"], metadata["Longitude"], metadata["Time Zone"], metadata["Elevation"], metadata["Local Time Zone"]),
        'Year,Month,Day,Hour,Minute,DNI,DHI,GHI,Dew Point,Temperature,Pressure,Wind Direction,Wind Speed,Surface Albedo\n',
    ]
    header_str = ''.join(header)
    print(f"Debug: Reservoir name inside parse_psm3: {reservoir_name}")
    # Write SAM-compatible file

    # Your desired directory
    directory_path = "/content/drive/MyDrive/Colombia_nrel_all_location/"
    # Updated SAM-compatible file path
    sam_weather_file = f"{directory_path}sam_weather_file_{reservoir_name}_{lat}_{lon}.csv"

    #sam_weather_file = f"sam_weather_file_{reservoir_name}+_{lat}_+{lon}.csv"
    with open(sam_weather_file, 'w') as sam_file:
        sam_file.write(header_str)
        data.to_csv(sam_file, index=False, header=False, sep=',')

    return data, metadata, sam_weather_file


def download_weather_data(lat, lon, reservoir_name):
    #dataset = "2021"  ## for southern brazil only 2019,2021, 2022 dta available
    dataset="tmy-2021"
    '''
    if lat>=-21.186575058950382:
      dataset = "tmy-2021"
    else:
      dataset = "2021"
    '''
    #api_key = "qguVH9fdgUOyRo1jo6zzOUXkS6a96vY1ct45RpuK"
    #url = f"https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?wkt=POINT({lon}%20{lat})&names={dataset}&leap_day=false&interval=60&utc=false&full_name=Your%20Name&email=atiqureee@gmail.com&affiliation=Your%20Organization&mailing_list=true&reason=example&api_key={api_key}&attributes=dhi,dni,ghi,clearsky_dhi,clearsky_dni,clearsky_ghi,cloud_type,dew_point,air_temperature,surface_albedo,surface_pressure,relative_humidity,temperature_2m_nwp,precipitable_water,wind_direction_10m_nwp,wind_speed_10m_nwp,wind_speed_100m_nwp"

    #response = requests.get(url)
    #response.raise_for_status()

    #return StringIO(response.text)
    email='atiqureee@gmail.com'
    NREL_API_KEY='qguVH9fdgUOyRo1jo6zzOUXkS6a96vY1ct45RpuK'

    weather, metadata, sam_weather_file = get_psm3(lat, lon, NREL_API_KEY, email, names=dataset, interval=60,
                                                   attributes=('air_temperature', 'dew_point', 'dhi', 'dni', 'ghi',
                                                               'surface_albedo', 'surface_pressure', 'wind_direction',
                                                               'wind_speed'), leap_day=False, full_name='pvlib python',
                                                   affiliation='pvlib python', map_variables=None, timeout=60, reservoir_name=reservoir_name)
    print(f"Debug: Reservoir name inside download_weather_data: {reservoir_name}")
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
    temp_file.close()
    weather.to_csv(temp_file.name, index=False)

    # Create a new "Time" column using the "Year," "Month," "Day," and "Hour" columns
    weather["Time"] = pd.to_datetime(weather[["Year", "Month", "Day", "Hour"]])

    # Calculate average monthly albedo values
    weather["Month"] = pd.to_datetime(weather["Time"]).dt.month
    albedo_column = "albedo" if "albedo" in weather.columns else "Surface Albedo"
    monthly_albedo = weather.groupby("Month")[albedo_column].mean().tolist()


    return sam_weather_file, monthly_albedo, weather, metadata



def sapm_cell_FPV(poa_global, temp_air, wind_speed, a, b, deltaT,
              irrad_ref=1000.):

    module_temperature = sapm_module_FPV(poa_global, temp_air, wind_speed,
                                     a, b)
    return sapm_cell_from_module_FPV(module_temperature, poa_global, deltaT,
                                 irrad_ref)


def sapm_module_FPV(poa_global, temp_air, wind_speed, a, b):

    return poa_global * np.exp(a + b * wind_speed) + temp_air

def sapm_cell_from_module_FPV(module_temperature, poa_global, deltaT,
                          irrad_ref=1000.):

    return module_temperature + (poa_global / irrad_ref) * deltaT

def pvsyst_cell_fpv(poa_global, temp_air, wind_speed=1.0, u_c=29.0, u_v=0.0,
                eta_m=None, module_efficiency=0.1, alpha_absorption=0.9):

    if eta_m:
        warn_deprecated(
            since='v0.9', message='eta_m overwriting module_efficiency',
            name='eta_m', alternative='module_efficiency', removal='v0.10')
        module_efficiency = eta_m
    total_loss_factor = u_c + u_v * wind_speed
    heat_input = poa_global * alpha_absorption * (1 - module_efficiency)
    temp_difference = heat_input / total_loss_factor
    return temp_air + temp_difference

def pvsyst_cell_pv(poa_global, temp_air, wind_speed=1.0, u_c=29.0, u_v=0.0,
                eta_m=None, module_efficiency=0.1, alpha_absorption=0.9):

    if eta_m:
        warn_deprecated(
            since='v0.9', message='eta_m overwriting module_efficiency',
            name='eta_m', alternative='module_efficiency', removal='v0.10')
        module_efficiency = eta_m
    total_loss_factor = u_c + u_v * wind_speed
    heat_input = poa_global * alpha_absorption * (1 - module_efficiency)
    temp_difference = heat_input / total_loss_factor
    return temp_air + temp_difference


# Read the .xlsx file
filename = '/content/Main_ALL_colombia_qgis_DDSA_OSM_v27_colabmerged.xlsx'
data = pd.read_excel(filename, sheet_name='calculation all important ')
output_csv = "output.csv"
# Conversion factor for ET to E
conversion_factor_et_to_e = 0.8
# DataFrames to store the results
evaporation_df = pd.DataFrame()
rmse_results_E = {}
# DataFrames to store the results
evaporation_df = pd.DataFrame()

# Assuming 'data' is your dataframe with 440 rows of lat_osm, lon_osm, area_osm, and ID

# Daily evaporation dataframe for the first 29 rows
# Aggregate evaporation dataframe for all 440 locations
aggregate_evaporation_columns = ['ID', 'lat_osm', 'lon_osm', 'avg_daily_evaporation', 'total_yearly_evaporation', 'evaporation_saved_by_fcover_m3']
aggregate_evaporation_df = pd.DataFrame(columns=aggregate_evaporation_columns)


#data = pd.read_csv(input_csv)
data["PV annual energy (kWh)"] = 0.0

for index, row in data.iterrows():
        print(f"Running case number: {index} grid: {index}")

        lat = row['latitude_all']
        lon = row["longitude_all"]
        ID=row["ID_ALL"]
        ReservoirArea = row['Reservoir_area_all_km2']
        reservoir_name = f"{ID}_"
        PanelArea = 0.3 * ReservoirArea

        if PanelArea > 6:
          PanelArea = 6
        #PanelArea = row["SURFACE_AREA"]
        #area_km2 = row["SURFACE_AREA"]
        #pv_kw = 0.3 * area_acres * (1 / 6) * 1000  # for every 1MW there are 6 acres land requires but we chose 2.5


        #area_acres = PanelArea * 247.105  # Convert km² to acres
        #pv_kw = area_acres * (1 / 2.5) * 1000 # thye have given total areas 30% we already did that so area comes to acres

        module_area_m2_fromtotal_PVarea=(771/1265)*PanelArea*1000000
        sam_weather_file_data, albedo_data, weather_full_dataframe, metadata  = download_weather_data(lat, lon, reservoir_name)
        # Set the Time column as the index
        for col in ['ghi', 'dni', 'dhi']:
          weather_full_dataframe[f"{col}_MJ_m2"] = weather_full_dataframe[col] * 3600 / 10**6

        # Set the Time column as the index
        weather_full_dataframe.set_index('Time', inplace=True)

        # Resample to daily frequency
        weather_daily = weather_full_dataframe.resample('D').agg({
            'Year': 'first',
            'Month': 'first',
            'Day': 'first',
            'ghi_MJ_m2': 'sum',
            'dni_MJ_m2': 'sum',
            'dhi_MJ_m2': 'sum',
            'temp_air': ['mean', 'max', 'min'],  # mean, max, and min temperature
            'wind_speed': 'mean',
            'pressure': 'mean',
            'Dew Point': 'mean',
            'wind_direction': 'mean',
            'albedo': 'mean'
        })

        # Flatten the multi-level columns
        weather_daily.columns = [' '.join(col).strip() for col in weather_daily.columns.values]

        # Rename columns for max and min temperature
        weather_daily.rename(columns={'temp_air max': 'Ta. Max.', 'temp_air min': 'Ta. Min.'}, inplace=True)

        # Reset index
        weather_daily.reset_index(drop=True, inplace=True)


        #pv_system = pvsam.new()
        #pv_system = pvsam.default("FlatPlatePVCommercial")
        #evaporation determination
        data_sheet = copy.deepcopy(weather_daily)

        # Read the original Excel file to get the original values
        original_data = copy.deepcopy(data_sheet)
        total_average_reservoir_area_km2 = ReservoirArea # in km²
        fpv_area = PanelArea*1000000  # in m2
        fpv_openings = (199.81/1265)* fpv_area  # in m2
        total_reservoir_area_m2 = total_average_reservoir_area_km2 * 1e6
        coverage_ratio=(fpv_area-fpv_openings)/total_reservoir_area_m2
        #ghi
        data_sheet["Rs (MJ/m²d) solar irrdaiance"]=copy.deepcopy(data_sheet['ghi_MJ_m2 sum'])
        # Calculations
        data_sheet["Rns (MJ/m²d)"] = (1 - data_sheet["albedo mean"]) * data_sheet["Rs (MJ/m²d) solar irrdaiance"]
        data_sheet["φ Dec."] = -25.50
        data_sheet["φ Rad. (latitude)"] = (pi / 180) * lat
        data_sheet["dr (rad) (inverse relative distance Earth-Sun)"] = 1 + 0.033 * np.cos(2 * pi / 365 * data_sheet["Day first"])
        data_sheet["δ (rad) solar declination"] = 0.409 * np.sin((2 * pi * data_sheet["Day first"] / 365) - 1.39)
        data_sheet["ωs (sunset hour angle)"] = np.arccos(-np.tan(data_sheet["φ Rad. (latitude)"]) * np.tan(data_sheet["δ (rad) solar declination"]))

        data_sheet["sin(φ)*sin(δ)"] = np.sin(data_sheet["φ Rad. (latitude)"]) * np.sin(data_sheet["δ (rad) solar declination"])
        data_sheet["cos(φ)*cos(δ)"] = np.cos(data_sheet["φ Rad. (latitude)"]) * np.cos(data_sheet["δ (rad) solar declination"])
        data_sheet["Ra(MJ m-2d-1)"] = 37.586 * data_sheet["dr (rad) (inverse relative distance Earth-Sun)"] * (
            data_sheet["sin(φ)*sin(δ)"] + data_sheet["cos(φ)*cos(δ)"] * np.sin(data_sheet["ωs (sunset hour angle)"]))
        data_sheet["N (h)"] = 24 / pi * data_sheet["ωs (sunset hour angle)"]
        data_sheet["Rso(MJ m-2d-1)"] = (0.75 + (0.00002 * metadata['Elevation'])) * data_sheet["Ra(MJ m-2d-1)"]
        data_sheet["σ(MJm²dia-1)"] = 4.903e-9
        data_sheet["Ta. Av."]=data_sheet['temp_air mean']

        data_sheet["ea (kPa)"] = 0.6108 * np.exp((17.27 * data_sheet["Dew Point mean"]) / (237.3 + data_sheet["Dew Point mean"]))
        data_sheet["es.max (kPa)"] = 0.6108 * np.exp(17.3 * data_sheet["Ta. Max."] / (237.3 + data_sheet["Ta. Max."]))
        data_sheet["es.min (kPa)"] = 0.6108 * np.exp(17.3 * data_sheet["Ta. Min."] / (237.3 + data_sheet["Ta. Min."]))
        data_sheet["es (kPa)"] = (data_sheet["es.max (kPa)"] + data_sheet["es.min (kPa)"]) / 2

        #data_sheet["Rnl(MJ m-2d-1)"]  =   (1.35 * np.minimum(data_sheet["Rs (MJ/m²d) solar irrdaiance"] / data_sheet["Rso(MJ m-2d-1)"], 1) - 0.35)*(0.34 - 0.14 * np.sqrt(data_sheet["ea (kPa)"])) * data_sheet["σ(MJm²dia-1)"] * (data_sheet["Ta. Av."] + 273.2)**4
        #data_sheet["Rnl(MJ m-2d-1)"]  =   (1.35 * (data_sheet["Rs (MJ/m²d) solar irrdaiance"] / data_sheet["Rso(MJ m-2d-1)"]) - 0.35)*(0.34 - 0.14 * np.sqrt(data_sheet["ea (kPa)"])) * data_sheet["σ(MJm²dia-1)"] * (data_sheet["Ta. Av."] + 273.2)**4

        data_sheet["Rnl(MJ m-2d-1)"] = data_sheet["σ(MJm²dia-1)"] * ((data_sheet["Ta. Max."] + 273.16)**4 + (data_sheet["Ta. Min."] + 273.16)**4 / 2)* (0.34 - 0.14 * np.sqrt(data_sheet["ea (kPa)"])) * (1.35 * (data_sheet["Rs (MJ/m²d) solar irrdaiance"] / data_sheet["Rso(MJ m-2d-1)"]) - 0.35)

        data_sheet["Rn (MJ m-2d-1)"] = data_sheet["Rns (MJ/m²d)"] - data_sheet["Rnl(MJ m-2d-1)"]

        data_sheet["Rns_fcover (MJ/m²d)"]  = 0
        data_sheet["Rnl_fcover(MJ m-2d-1)"]= data_sheet["σ(MJm²dia-1)"] * ((data_sheet["Ta. Max."] + 273.16)**4 + (data_sheet["Ta. Min."] + 273.16)**4 / 2)* (0.34 - 0.14 * np.sqrt(data_sheet["ea (kPa)"])) * (0.1)
        data_sheet["Rn_fcover (MJ m-2d-1)"] = data_sheet["Rns_fcover (MJ/m²d)"] - data_sheet["Rnl_fcover(MJ m-2d-1)"]

        data_sheet["λ (MJ.kg-1)"] = 2.501 - data_sheet["Ta. Av."] * (2.361e-3)
        data_sheet["Patm(kPa)"] = 101.3 * (((293 - (0.0065 * metadata['Elevation'])) / 293) ** 5.26)
        data_sheet["γ(kPa/ºC)"] = (0.665e-3) * data_sheet["Patm(kPa)"]


        # no humdity is here so
        #data_sheet["ea (kPa)"] = (data_sheet["es (kPa)"] * data_sheet["U.avg.%"]) / 100


        data_sheet["dew point (C)"] = (237.3 * np.log10(data_sheet["ea (kPa)"] / 0.6108)) / (7.5 - np.log10(data_sheet["ea (kPa)"] / 0.6108))

        data_sheet["ed (kpa)"] = 0.6108 * np.exp(17.3 * data_sheet["dew point (C)"] / (237.3 + data_sheet["dew point (C)"]))

        data_sheet["Δ (kPa/ºC)"] = 4098 * ((0.6108 * np.exp(17.3 * data_sheet["Ta. Av."] / (237.3 + data_sheet["Ta. Av."]))) / ((data_sheet["Ta. Av."] + 273.2) ** 2))

        data_sheet["D (kPa)"] = data_sheet["es (kPa)"] - data_sheet["ea (kPa)"]
        #data_sheet["D (kPa)"] = data_sheet["D (kPa)"].apply(lambda x: max(0, x))

        ## penmen monteith ((0.408 * AE2 * W2) + ((Y2 * 900 * G2 * (AB2 - AD2)) / (C2 + 273))) / (AE2 + Y2 * (1 + 0.34 * G2))
        data_sheet["Penman-Monteith equation ET (mm/day) (2021)"] = ((0.408 * data_sheet["Δ (kPa/ºC)"] * data_sheet["Rn (MJ m-2d-1)"]) + ((data_sheet["γ(kPa/ºC)"] * 900 * data_sheet["wind_speed mean"] * (data_sheet["D (kPa)"])) / (data_sheet["Ta. Av."] + 273))) / (data_sheet["Δ (kPa/ºC)"] + data_sheet["γ(kPa/ºC)"] * (1 + 0.34 * data_sheet["wind_speed mean"]))
        # Rower equation ET (mm/day) (2021-2022)

        data_sheet["Penman-Monteith equation ET (mm/day) _method3_fcover(2021)"]= ((0.408 * data_sheet["Δ (kPa/ºC)"] *(data_sheet["Rn (MJ m-2d-1)"] * (1 - coverage_ratio) + data_sheet["Rn_fcover (MJ m-2d-1)"] * coverage_ratio)) + ((data_sheet["γ(kPa/ºC)"] * 900 * data_sheet["wind_speed mean"] * (data_sheet["D (kPa)"])) / (data_sheet["Ta. Av."] + 273))) / (data_sheet["Δ (kPa/ºC)"]+ data_sheet["γ(kPa/ºC)"] * (1 + 0.34 * data_sheet["wind_speed mean"]))


        ## evaporation
        surface_area_km2 = total_average_reservoir_area_km2
        surface_area_m2 = surface_area_km2 * 1e6
        effective_open_area_m2 = (surface_area_m2 - fpv_area) + fpv_openings
        sliced_method_data = data_sheet["Penman-Monteith equation ET (mm/day) (2021)"] * conversion_factor_et_to_e
        free_evaporation_hm3 = (sliced_method_data / 1000) * surface_area_km2

        fcover_evaporation_hm3 = (data_sheet["Penman-Monteith equation ET (mm/day) _method3_fcover(2021)"]* conversion_factor_et_to_e / 1000) * effective_open_area_m2 / 1e6

        savings_method_3_m3 = (free_evaporation_hm3.sum()- fcover_evaporation_hm3.sum())*1e6

        # Other calculations for water savings methods 1, 2, 3...
        if index < 29:
          evaporation_df[f"{reservoir_name} free evaporation E mm/day"] = sliced_method_data
          evaporation_df[f"{reservoir_name} free evaporation (m3)"] = free_evaporation_hm3*1e6
          evaporation_df[f"{reservoir_name} fcover evaporation using water savings method 3 E mm/day"] = data_sheet["Penman-Monteith equation ET (mm/day) _method3_fcover(2021)"]* conversion_factor_et_to_e
          evaporation_df[f"{reservoir_name} fcover evaporation using water savings method 3 E m3/day"]=fcover_evaporation_hm3*1e6
          #evaporation_df[f"{reservoir_name} water savings by method 3 (m3)"] =savings_method_3_m3
            # Compute aggregate values for all locations
        avg_daily_evaporation = sliced_method_data.mean()
        total_yearly_evaporation = sliced_method_data.sum()
        evaporation_saved_by_fcover_m3 = savings_method_3_m3

        aggregate_evaporation_df.loc[index] = [ID, lat, lon, avg_daily_evaporation, total_yearly_evaporation, evaporation_saved_by_fcover_m3]

# Truncate lat_osm and lon_osm in aggregate_evaporation_df

# Save dataframes to files (compatible with Google Colab)
evaporation_df.to_csv('/content/drive/MyDrive/Colombia_nrel_all_location/daily_evaporation_colombia_v27.csv', index=False)
aggregate_evaporation_df.to_csv('/content/drive/MyDrive/Colombia_nrel_all_location/ALL_aggregate_evaporation_colombia_v27.csv', index=False)




# Adding Penman-Monteith Method 3 to evaporation_df
# Code for Penman-Monteith Method 3...

# Displaying the evaporation DataFrame
evaporation_df.head()



Running case number: 0 grid: 0
Debug: Reservoir name inside parse_psm3: 312_
Debug: Reservoir name inside download_weather_data: 312_
Running case number: 1 grid: 1
Debug: Reservoir name inside parse_psm3: 279_
Debug: Reservoir name inside download_weather_data: 279_
Running case number: 2 grid: 2
Debug: Reservoir name inside parse_psm3: 113_
Debug: Reservoir name inside download_weather_data: 113_
Running case number: 3 grid: 3
Debug: Reservoir name inside parse_psm3: 426_
Debug: Reservoir name inside download_weather_data: 426_
Running case number: 4 grid: 4
Debug: Reservoir name inside parse_psm3: 303_
Debug: Reservoir name inside download_weather_data: 303_
Running case number: 5 grid: 5
Debug: Reservoir name inside parse_psm3: 117_
Debug: Reservoir name inside download_weather_data: 117_
Running case number: 6 grid: 6
Debug: Reservoir name inside parse_psm3: 298_
Debug: Reservoir name inside download_weather_data: 298_
Running case number: 7 grid: 7
Debug: Reservoir name inside pa

<ipython-input-9-334ccbd3ee26>:461: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  evaporation_df[f"{reservoir_name} free evaporation E mm/day"] = sliced_method_data
<ipython-input-9-334ccbd3ee26>:462: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  evaporation_df[f"{reservoir_name} free evaporation (m3)"] = free_evaporation_hm3*1e6
<ipython-input-9-334ccbd3ee26>:463: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

Running case number: 26 grid: 26
Debug: Reservoir name inside parse_psm3: 5_
Debug: Reservoir name inside download_weather_data: 5_


<ipython-input-9-334ccbd3ee26>:461: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  evaporation_df[f"{reservoir_name} free evaporation E mm/day"] = sliced_method_data
<ipython-input-9-334ccbd3ee26>:462: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  evaporation_df[f"{reservoir_name} free evaporation (m3)"] = free_evaporation_hm3*1e6
<ipython-input-9-334ccbd3ee26>:463: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

Running case number: 27 grid: 27
Debug: Reservoir name inside parse_psm3: 438_
Debug: Reservoir name inside download_weather_data: 438_


<ipython-input-9-334ccbd3ee26>:461: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  evaporation_df[f"{reservoir_name} free evaporation E mm/day"] = sliced_method_data
<ipython-input-9-334ccbd3ee26>:462: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  evaporation_df[f"{reservoir_name} free evaporation (m3)"] = free_evaporation_hm3*1e6
<ipython-input-9-334ccbd3ee26>:463: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

Running case number: 28 grid: 28
Debug: Reservoir name inside parse_psm3: 123_
Debug: Reservoir name inside download_weather_data: 123_


<ipython-input-9-334ccbd3ee26>:461: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  evaporation_df[f"{reservoir_name} free evaporation E mm/day"] = sliced_method_data
<ipython-input-9-334ccbd3ee26>:462: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  evaporation_df[f"{reservoir_name} free evaporation (m3)"] = free_evaporation_hm3*1e6
<ipython-input-9-334ccbd3ee26>:463: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

Running case number: 29 grid: 29
Debug: Reservoir name inside parse_psm3: 427_
Debug: Reservoir name inside download_weather_data: 427_
Running case number: 30 grid: 30
Debug: Reservoir name inside parse_psm3: 14_
Debug: Reservoir name inside download_weather_data: 14_
Running case number: 31 grid: 31
Debug: Reservoir name inside parse_psm3: 255_
Debug: Reservoir name inside download_weather_data: 255_
Running case number: 32 grid: 32
Debug: Reservoir name inside parse_psm3: 337_
Debug: Reservoir name inside download_weather_data: 337_
Running case number: 33 grid: 33
Debug: Reservoir name inside parse_psm3: 149_
Debug: Reservoir name inside download_weather_data: 149_
Running case number: 34 grid: 34
Debug: Reservoir name inside parse_psm3: 292_
Debug: Reservoir name inside download_weather_data: 292_
Running case number: 35 grid: 35
Debug: Reservoir name inside parse_psm3: 6_
Debug: Reservoir name inside download_weather_data: 6_
Running case number: 36 grid: 36
Debug: Reservoir name

,312_ free evaporation E mm/day,312_ free evaporation (m3),312_ fcover evaporation using water savings method 3 E mm/day,312_ fcover evaporation using water savings method 3 E m3/day,279_ free evaporation E mm/day,279_ free evaporation (m3),279_ fcover evaporation using water savings method 3 E mm/day,279_ fcover evaporation using water savings method 3 E m3/day,113_ free evaporation E mm/day,113_ free evaporation (m3),...,5_ fcover evaporation using water savings method 3 E mm/day,5_ fcover evaporation using water savings method 3 E m3/day,438_ free evaporation E mm/day,438_ free evaporation (m3),438_ fcover evaporation using water savings method 3 E mm/day,438_ fcover evaporation using water savings method 3 E m3/day,123_ free evaporation E mm/day,123_ free evaporation (m3),123_ fcover evaporation using water savings method 3 E mm/day,123_ fcover evaporation using water savings method 3 E m3/day
0,2.718523,187436.828525,2.581200,164927.774012,3.281207,211076.130576,3.017053,178840.401141,2.774616,171826.648304,...,2.037514,3761.611465,2.624188,3181.879955,2.076121,1881.422387,2.429210,2739.881461,1.777888,1498.704051
1,2.835023,195469.294976,2.690990,171942.838648,3.560053,229013.917209,3.273955,194068.614575,2.821514,174730.963793,...,2.076740,3834.029674,3.037774,3683.361957,2.447966,2218.396080,2.227960,2512.894321,1.632415,1376.075081
2,2.346894,161813.772423,2.215308,141548.779875,3.084595,198428.273956,2.835501,168078.642836,2.553058,158106.017594,...,1.873179,3458.220094,3.131869,3797.453705,2.565150,2324.590649,2.545503,2871.047195,1.870926,1577.132572
3,2.402364,165638.291625,2.280513,145715.132905,2.695599,173404.644694,2.475979,146767.411474,3.588738,222243.690993,...,2.649657,4891.736145,3.943314,4781.347092,3.206238,2905.557380,2.031360,2291.150649,1.480467,1247.987905
4,2.701729,186278.967353,2.565301,163911.839768,2.404699,154691.388307,2.208086,130887.621539,3.327768,206082.298243,...,2.461257,4543.917448,2.352025,2851.877812,1.918897,1738.943659,2.882773,3251.450291,2.120025,1787.114860


In [ ]:
evaporation_df.to_csv("all_southernbrazil_selectedreservoir_evaporation_v27_10pct_6km2.csv")

In [ ]:
evaporation_df.to_csv('/content/drive/MyDrive/Colombia_nrel_all_location/daily_evaporation_colombia_v27.csv', index=False)
aggregate_evaporation_df.to_csv('/content/drive/MyDrive/Colombia_nrel_all_location/ALL_aggregate_evaporation_colombia_v27.csv', index=False)




### Energy generation and economic evaluation PVlib

In [ ]:
import io
import requests
import pandas as pd
import pvlib
from json import JSONDecodeError
import warnings
from pvlib._deprecation import pvlibDeprecationWarning
import numpy as np
import numpy_financial as npf
import datetime
import xarray as xr
import math
#import cupy as cp
from timezonefinder import TimezoneFinder
from pvlib.tools import sind
from pvlib._deprecation import warn_deprecated
from pvlib.tools import _get_sample_intervals
import scipy
import scipy.constants
import warnings
import json
import PySAM.Pvsamv1 as pvsam
from PySAM import Utilityrate5 as ur5  # Add this import statement
#import pysam
import os
from io import StringIO
import copy
import tempfile
from math import pi, cos, sin, tan, acos, sqrt, log, exp




# 'relative_humidity', 'total_precipitable_water' are not available
ATTRIBUTES = (
    'air_temperature', 'dew_point', 'dhi', 'dni', 'ghi', 'surface_albedo',
    'surface_pressure', 'wind_direction', 'wind_speed')
PVLIB_PYTHON = 'pvlib python'

# Dictionary mapping PSM3 names to pvlib names
VARIABLE_MAP = {
    'GHI': 'ghi',
    'DHI': 'dhi',
    'DNI': 'dni',
    'Clearsky GHI': 'ghi_clear',
    'Clearsky DHI': 'dhi_clear',
    'Clearsky DNI': 'dni_clear',
    'Solar Zenith Angle': 'solar_zenith',
    'Temperature': 'temp_air',
    'Relative Humidity': 'relative_humidity',
    'Dew point': 'temp_dew',
    'Pressure': 'pressure',
    'Wind Direction': 'wind_direction',
    'Wind Speed': 'wind_speed',
    'Surface Albedo': 'albedo',
    'Precipitable Water': 'precipitable_water',
}


def get_psm3(latitude, longitude, api_key, email, names='tmy', interval=60,
             attributes=ATTRIBUTES, leap_day=None, full_name=PVLIB_PYTHON,
             affiliation=PVLIB_PYTHON, map_variables=None, timeout=30, reservoir_name=""):
    # The well know text (WKT) representation of geometry notation is strict.
    # A POINT object is a string with longitude first, then the latitude, with
    # four decimals each, and exactly one space between them.
    longitude = ('%9.4f' % longitude).strip()
    latitude = ('%8.4f' % latitude).strip()
    # TODO: make format_WKT(object_type, *args) in tools.py

    # convert to string to accomodate integer years being passed in
    names = str(names)

    # convert pvlib names in attributes to psm3 convention (reverse mapping)
    # unlike psm3 columns, attributes are lower case and with underscores
    amap = {value: key.lower().replace(' ', '_') for (key, value) in
            VARIABLE_MAP.items()}
    attributes = [amap.get(a, a) for a in attributes]
    attributes = list(set(attributes))  # remove duplicate values

    if (leap_day is None) and (not names.startswith('t')):
        warnings.warn(
            'The ``get_psm3`` function will default to leap_day=True '
            'starting in pvlib 0.11.0. Specify leap_day=True '
            'to enable this behavior now, or specify leap_day=False '
            'to hide this warning.', pvlibDeprecationWarning)
        leap_day = False

    # required query-string parameters for request to PSM3 API
    params = {
        'api_key': api_key,
        'full_name': full_name,
        'email': email,
        'affiliation': affiliation,
        'reason': PVLIB_PYTHON,
        'mailing_list': 'false',
        'wkt': 'POINT(%s %s)' % (longitude, latitude),
        'names': names,
        'attributes':  ','.join(attributes),
        'leap_day': str(leap_day).lower(),
        'utc': 'false',
        'interval': interval
    }
    NSRDB_API_BASE = "https://developer.nrel.gov"
    PSM_URL1 = NSRDB_API_BASE + "/api/nsrdb/v2/solar/psm3-download.csv"
    TMY_URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/psm3-tmy-download.csv"
    PSM5MIN_URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/psm3-5min-download.csv"

    # First, check longitude conditions
    if -16 < lon < 91:
        PSM_URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/msg-iodc-download.csv"
    elif 91 < lon < 182:
        PSM_URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/himawari-download.csv"
    else:
        # Then check latitude conditions if longitude didn't match
        if lat >= -21.186575058950382:
            PSM_URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/psm3-2-2-tmy-download.csv"
        else:
            PSM_URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/full-disc-download.csv"

    # Finally, set URL based on interval and 'names' prefix
    if any(prefix in names for prefix in ('tmy', 'tgy', 'tdy')):
        URL = NSRDB_API_BASE + "/api/nsrdb/v2/solar/psm3-2-2-tmy-download.csv"
    elif interval in (5, 15):
        URL = PSM5MIN_URL
    else:
        URL = PSM_URL

    response = requests.get(URL, params=params, timeout=timeout)

    if not response.ok:
        # if the API key is rejected, then the response status will be 403
        # Forbidden, and then the error is in the content and there is no JSON
        try:
            errors = response.json()['errors']
        except JSONDecodeError:
            errors = response.content.decode('utf-8')
        raise requests.HTTPError(errors, response=response)
    # the CSV is in the response content as a UTF-8 bytestring
    # to use pandas we need to create a file buffer from the response
    fbuf = io.StringIO(response.content.decode('utf-8'))
    return parse_psm3(fbuf, map_variables, reservoir_name)



def parse_psm3(fbuf, map_variables=None, reservoir_name=""):
    # Read metadata
    metadata_fields = fbuf.readline().split(',')
    metadata_fields[-1] = metadata_fields[-1].strip()
    metadata_values = fbuf.readline().split(',')
    metadata_values[-1] = metadata_values[-1].strip()
    metadata = dict(zip(metadata_fields, metadata_values))

    # Set some metadata types to numbers
    metadata['Local Time Zone'] = int(metadata['Local Time Zone'])
    metadata['Time Zone'] = int(metadata['Time Zone'])
    metadata['Latitude'] = float(metadata['Latitude'])
    metadata['Longitude'] = float(metadata['Longitude'])
    metadata['Elevation'] = int(metadata['Elevation'])

    # Read weather data
    columns = fbuf.readline().split(',')
    columns[-1] = columns[-1].strip()
    columns = [col for col in columns if col != '']
    dtypes = dict.fromkeys(columns, float)
    dtypes.update(Year=int, Month=int, Day=int, Hour=int, Minute=int)
    dtypes['Cloud Type'] = int
    dtypes['Fill Flag'] = int
    data = pd.read_csv(
        fbuf, header=None, names=columns, usecols=columns, dtype=dtypes,
        delimiter=',', lineterminator='\n')

    # Convert date vector to datetime
    dtidx = pd.to_datetime(
        data[['Year', 'Month', 'Day', 'Hour', 'Minute']])
    tz = 'Etc/GMT%+d' % -metadata['Time Zone']
    data.index = pd.DatetimeIndex(dtidx).tz_localize(tz)

    # Rename PSM3 variable names to SAM-compatible names
    data = data.rename(columns=VARIABLE_MAP)
    # Rearrange columns to match the expected order
    #data = data[['Year', 'Month', 'Day', 'Hour', 'Minute', 'DNI', 'DHI', 'GHI', 'Dew Point', 'Temperature', 'Pressure', 'Wind Direction', 'Wind Speed', 'Surface Albedo']]
    data=data[['Year', 'Month', 'Day', 'Hour', 'Minute', 'dni', 'dhi', 'ghi', 'Dew Point', 'temp_air', 'pressure', 'wind_direction', 'wind_speed', 'albedo']]
    #Year	Month	Day	Hour	Minute	wind_speed	dhi	ghi	pressure	wind_direction	albedo	dni	temp_air	Dew Point

    # Add a "Timestamp" column
    #data['Timestamp'] = data.index.strftime('%Y%m%d:%H%M')

    # Create SAM-compatible file header
    header = [
        'Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,Dew Point Units,DHI Units,DNI Units,GHI Units,Temperature Units,Pressure Units,Wind Direction Units,Wind Speed Units,Surface Albedo Units,Version\n',
        'NSRDB,{},-,-,-,{},{},{},{},{},C,w/m2,w/m2,w/m2,C,mbar,Degrees,m/s,N/A,3.1.0\n'.format(metadata["Location ID"], metadata["Latitude"], metadata["Longitude"], metadata["Time Zone"], metadata["Elevation"], metadata["Local Time Zone"]),
        'Year,Month,Day,Hour,Minute,DNI,DHI,GHI,Dew Point,Temperature,Pressure,Wind Direction,Wind Speed,Surface Albedo\n',
    ]
    header_str = ''.join(header)
    print(f"Debug: Reservoir name inside parse_psm3: {reservoir_name}")
    # Write SAM-compatible file
    sam_weather_file = f"sam_weather_file_{reservoir_name}+_{lat}_+{lon}.csv"
    with open(sam_weather_file, 'w') as sam_file:
        sam_file.write(header_str)
        data.to_csv(sam_file, index=False, header=False, sep=',')

    return data, metadata, sam_weather_file


def download_weather_data(lat, lon, reservoir_name):
    dataset = "2021"  ## fro souther brazil only 2019,2021, 2022 dta available
    '''
    if lat>=-21.186575058950382:
      dataset = "tmy-2021"
    else:
      dataset = "2021"
    '''
    #api_key = "qguVH9fdgUOyRo1jo6zzOUXkS6a96vY1ct45RpuK"
    #url = f"https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?wkt=POINT({lon}%20{lat})&names={dataset}&leap_day=false&interval=60&utc=false&full_name=Your%20Name&email=atiqureee@gmail.com&affiliation=Your%20Organization&mailing_list=true&reason=example&api_key={api_key}&attributes=dhi,dni,ghi,clearsky_dhi,clearsky_dni,clearsky_ghi,cloud_type,dew_point,air_temperature,surface_albedo,surface_pressure,relative_humidity,temperature_2m_nwp,precipitable_water,wind_direction_10m_nwp,wind_speed_10m_nwp,wind_speed_100m_nwp"

    #return StringIO(response.text)
    email='atiqureee@gmail.com'
    NREL_API_KEY='qguVH9fdgUOyRo1jo6zzOUXkS6a96vY1ct45RpuK'

    weather, metadata, sam_weather_file = get_psm3(lat, lon, NREL_API_KEY, email, names=dataset, interval=60,
                                                   attributes=('air_temperature', 'dew_point', 'dhi', 'dni', 'ghi',
                                                               'surface_albedo', 'surface_pressure', 'wind_direction',
                                                               'wind_speed'), leap_day=False, full_name='pvlib python',
                                                   affiliation='pvlib python', map_variables=None, timeout=60, reservoir_name=reservoir_name)
    print(f"Debug: Reservoir name inside download_weather_data: {reservoir_name}")
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
    temp_file.close()
    weather.to_csv(temp_file.name, index=False)

    # Create a new "Time" column using the "Year," "Month," "Day," and "Hour" columns
    weather["Time"] = pd.to_datetime(weather[["Year", "Month", "Day", "Hour"]])

    # Calculate average monthly albedo values
    weather["Month"] = pd.to_datetime(weather["Time"]).dt.month
    albedo_column = "albedo" if "albedo" in weather.columns else "Surface Albedo"
    monthly_albedo = weather.groupby("Month")[albedo_column].mean().tolist()


    return sam_weather_file, monthly_albedo, weather, metadata



def sapm_cell_FPV(poa_global, temp_air, wind_speed, a, b, deltaT,
              irrad_ref=1000.):

    module_temperature = sapm_module_FPV(poa_global, temp_air, wind_speed,
                                     a, b)
    return sapm_cell_from_module_FPV(module_temperature, poa_global, deltaT,
                                 irrad_ref)


def sapm_module_FPV(poa_global, temp_air, wind_speed, a, b):

    return poa_global * np.exp(a + b * wind_speed) + temp_air

def sapm_cell_from_module_FPV(module_temperature, poa_global, deltaT,
                          irrad_ref=1000.):

    return module_temperature + (poa_global / irrad_ref) * deltaT

def pvsyst_cell_fpv(poa_global, temp_air, wind_speed=1.0, u_c=29.0, u_v=0.0,
                eta_m=None, module_efficiency=0.1, alpha_absorption=0.9):

    if eta_m:
        warn_deprecated(
            since='v0.9', message='eta_m overwriting module_efficiency',
            name='eta_m', alternative='module_efficiency', removal='v0.10')
        module_efficiency = eta_m
    total_loss_factor = u_c + u_v * wind_speed
    heat_input = poa_global * alpha_absorption * (1 - module_efficiency)
    temp_difference = heat_input / total_loss_factor
    return temp_air + temp_difference

def pvsyst_cell_pv(poa_global, temp_air, wind_speed=1.0, u_c=29.0, u_v=0.0,
                eta_m=None, module_efficiency=0.1, alpha_absorption=0.9):

    if eta_m:
        warn_deprecated(
            since='v0.9', message='eta_m overwriting module_efficiency',
            name='eta_m', alternative='module_efficiency', removal='v0.10')
        module_efficiency = eta_m
    total_loss_factor = u_c + u_v * wind_speed
    heat_input = poa_global * alpha_absorption * (1 - module_efficiency)
    temp_difference = heat_input / total_loss_factor
    return temp_air + temp_difference


# Read the .xlsx file
filename = '/content/drive/MyDrive/techno-economicanalysisFPV/selected reservoirs_southernbrazil_aug23_v26.xlsx'
data = pd.read_excel(filename)
output_csv = "output.csv"
# Conversion factor for ET to E
conversion_factor_et_to_e = 0.8
# DataFrames to store the results
evaporation_df = pd.DataFrame()
rmse_results_E = {}
# DataFrames to store the results
evaporation_df = pd.DataFrame()



# Add this function to get the timezone for given latitude and longitude
def get_timezone(latitude, longitude):
    from timezonefinder import TimezoneFinder
    tf = TimezoneFinder()
    return tf.timezone_at(lat=latitude, lng=longitude)

# Define location (latitude, longitude, and timezone)
#tz = 'America/Sao_Paulo'


# Define system parameters
surface_tilt = 15  # degrees
surface_azimuth = 0  # degrees (south)
albedo = 0.20  # Albedo for water surface

# Define PV module and inverter parameters (replace with your module and inverter specs)
#cec_module_data = pvlib.pvsystem.retrieve_sam('CECMod')
#cec_inverter_data = pvlib.pvsystem.retrieve_sam('CECInverter')
#module = cec_module_data['Canadian_Solar_Inc__CS6X_320P']
#inverter = cec_inverter_data['ABB__MICRO_0_25_I_OUTD_US_240__240V_']

sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
sandia_inverter = pvlib.pvsystem.retrieve_sam('SandiaInverter')
module = sandia_modules['Silevo_Triex_U300_Black__2014_']
inverter = sandia_inverter['Jiangsu_Zeversolar_New_Energy__Evershine_TL6000_US_240']
# Select the specific module and inverter
#print(sandia_inverters.columns[:5])
#print(sandia_modules.columns[:5])
#module = sandia_modules.loc[:, 523]
#inverter = sandia_inverters.loc[:, 1461]

# Economic parameters
system_cost_per_kw = 1000  # USD per kW
operation_maintenance_cost_per_kw = 20  # USD per kW per year
discount_rate = 0.04  # 7% discount rate
project_lifetime = 30  # years
electricity_price = 0.47  # USD per kWh

# Define specific power (W/m²) for the solar panels
specific_power = (module['Vmpo']*module['Impo'])/module['Area']
# Calculate NPV and LCOE for each row in the table

# Preallocate arrays for the results
num_rows = len(data.index)

#num_rows=1

Epen_all = np.zeros((2, 12, num_rows))
WS_all = np.zeros((12, num_rows))
# Preallocate arrays for the results
p_ac_all = np.zeros((12, num_rows))
p_ac_all_pv = np.zeros((12, num_rows))
npv_all = np.zeros(num_rows)
lcoe_all = np.zeros(num_rows)

p_ac_all_fpv_pv=np.zeros((2, 12, num_rows))


# Initialize the output DataFrame
#month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
#columns = ['reservoir_name', 'latitude', 'longitude', 'year', 'month', 'FPV_energy', 'PV_energy']
#monthly_data = pd.DataFrame(columns=columns)
column_names = ['reservoir_name', 'latitude', 'longitude', 'Annual_FPV_Energy_MWh', 'Annual_PV_Energy_MWh', 'state', 'reservoir_area','PanelArea_km2',]
# Add column names for each month
for i in range(1, 13):
    column_names.append(f'Month_{i}_FPV_Energy_kWh')
    column_names.append(f'Month_{i}_PV_Energy_kWh')

energy_data = pd.DataFrame(columns=column_names)


# Define your own loss percentages
my_soiling = 1
my_shading = 0  # No shading in your case, for example
my_snow = 0     # No snow in your case, for example
my_mismatch = 2
my_wiring = 2
my_connections = 0.5
my_lid = 2
my_nameplate_rating = 1
my_age = 0
my_availability = 1

# Calculate the total PVWatts loss factor
total_pvwatts_loss_factor = pvlib.pvsystem.pvwatts_losses(
    soiling=my_soiling, shading=my_shading, snow=my_snow,
    mismatch=my_mismatch, wiring=my_wiring, connections=my_connections,
    lid=my_lid, nameplate_rating=my_nameplate_rating, age=my_age,
    availability=my_availability
)





# Calculate the averages and store the results in the output DataFrame
# Define the number of days in each month
days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

ac_scaled_fpv_list=[]
ac_scaled_ground_pv_list=[]


#data = pd.read_csv(input_csv)
data["PV annual energy (kWh)"] = 0.0

for index, row in data.iterrows():
    print(f"Running case number: {index} grid: {index}")

    lat = row['latitude_all']
    lon = row["longitude_all"]
    ReservoirArea = row['Reservoir_area_all_km2']
    reservoir_name = row["Reservoir_name_all"]
    state1=row["State_twoword_all"]
    state2=row["Rservoir state_All"]
    PanelArea = 0.1 * ReservoirArea

    if PanelArea > 6:
      PanelArea = 6


    #area_acres = PanelArea * 247.105  # Convert km² to acres
    #pv_kw = area_acres * (1 / 2.5) * 1000 # thye have given total areas 30% we already did that so area comes to acres

    module_area_m2_fromtotal_PVarea=(771/1265)*PanelArea*1000000
    sam_weather_file_data, albedo_data, weather_full_dataframe, metadata  = download_weather_data(lat, lon, reservoir_name)

    tz = get_timezone(lat, lon)  # Get the corresponding timezone

    # Create a Location object
    location = pvlib.location.Location(lat, lon, tz)

    #system_size = specific_power * module_area_m2_fromtotal_PVarea
    #main_system_size_kW=system_size/1000

    # Calculate the array tilt
    #array_tilt = np.arctan(1.2 * ReservoirArea) * 180 / np.pi
    array_tilt = surface_tilt

    single_module_area = module['Area']  # area of a single module in m^2
    total_module_area = module_area_m2_fromtotal_PVarea  # total module area in m^2

    # Calculate number of modules in series and parallel
    #Array_Ms = np.floor(total_module_area / single_module_area)
    #Array_Mp = np.ceil(system_size / (module['Vmpo']*module['Impo']))

    '''
    # Calculate the area for a single module
    single_module_area = module['Area']  # area of a single module in m^2
    # Calculate the total module area based on available PanelArea
    total_module_area = module_area_m2_fromtotal_PVarea  # total module area in m^2
    # Calculate the number of modules that can fit into the total module area
    total_modules = np.floor(total_module_area / single_module_area)  # using floor to get an integer
    # Calculate the number of modules in parallel based on system size
    Array_Mp = np.ceil(main_system_size_kW * 1000 / (module['Vmpo'] * module['Impo']))  # main_system_size_kW is in kW, convert to W
    # Calculate the number of modules in series based on total modules and modules in parallel
    Array_Ms = np.floor(total_modules / Array_Mp)
    # Re-calculate the total modules to align with Array_Ms and Array_Mp
    total_modules = Array_Ms * Array_Mp
    # Re-calculate the system size based on the final total modules
    system_size= total_modules * module['Vmpo'] * module['Impo']
    main_system_size_kW= system_size/ 1000  # Convert to kW
    '''
    # Inverter specifications
    Vdcmax = inverter['Vdcmax']  # Maximum DC voltage
    Idcmax = inverter['Idcmax']   # Maximum DC current

    # Module specifications
    Vmpo = module['Vmpo']  # Maximum power voltage
    Impo = module['Impo']   # Maximum power current

    # Calculate number of modules in series (Ns)
    Array_Ms = np.floor(Vdcmax / Vmpo)

    # Calculate number of modules in parallel (Np)
    Array_Mp = np.floor(Idcmax / Impo)

    # Calculate total number of modules (N_total)
    total_modules = Array_Ms * Array_Mp

    #system_size= total_modules * module['Vmpo'] * module['Impo']
    #main_system_size_kW= system_size/ 1000  # Convert to kW



    # Calculate the total number of modules that can fit into the available area
    Total_Modules_Area = ReservoirArea * 1e6 / single_module_area  # Convert km^2 to m^2
    # Calculate the total system size based on the total module area
    total_system_size = Total_Modules_Area * module['Vmpo'] * module['Impo']
    # Calculate the number of inverters needed
    N_inverters = np.ceil(total_system_size / inverter['Paco'])
    # Your main system size in kW now becomes
    main_system_size_kW = total_system_size / 1000  # Convert to kW

    # Calculate solar position
    solpos = location.get_solarposition(weather_full_dataframe.index)

    # Calculate GHI
    #GHI_CERES_calc = weather_full_dataframe['dni'] * pvlib.tools.cosd(solpos['apparent_zenith']) + weather_full_dataframe['dhi']

    # Create the weather data DataFrame with the calculated GHI
    weather_data = pd.DataFrame({'ghi': weather_full_dataframe['ghi'], 'dni': weather_full_dataframe['dni'], 'dhi': weather_full_dataframe['dhi'], 'temp_air': weather_full_dataframe['temp_air'], 'wind_speed': weather_full_dataframe['wind_speed']}, index=weather_full_dataframe.index)

    # Calculate extraterrestrial radiation
    #dni_extra = pvlib.ir
    dni_extra = pvlib.irradiance.get_extra_radiation(solpos.index)
    # Calculate airmass
    airmass = location.get_airmass(solar_position=solpos)
    # Calculate angle of incidence
    aoi = pvlib.irradiance.aoi(array_tilt, surface_azimuth, solpos['apparent_zenith'], solpos['azimuth'])




    # Calculate total irradiance
    total_irradiance = pvlib.irradiance.get_total_irradiance(
        surface_tilt=array_tilt,
        surface_azimuth=surface_azimuth,
        solar_zenith=solpos['apparent_zenith'],
        solar_azimuth=solpos['azimuth'],
        dni=weather_data['dni'],
        ghi=weather_data['ghi'],
        dhi=weather_data['dhi'],
        dni_extra=dni_extra,
        airmass=airmass['airmass_relative'],
        albedo=albedo,
        model='perez',
    )





    # Calculate cell temperature
    cell_temperature_ground_pv = pvlib.temperature.sapm_cell(
    poa_global=total_irradiance['poa_global'],
    temp_air=weather_data['temp_air'],
    wind_speed=weather_data['wind_speed'],
    **pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass'],
        )


    # Calculate the cell temperature
    #temp_cell = pvsyst_cell_fpv(poa_global=total_irradiance['poa_global'], temp_air=ambient_temp, wind_speed=wind_speed, **module['temp_params'])
    temp_cell_fpv = pvsyst_cell_fpv(poa_global=total_irradiance['poa_global'], temp_air=weather_data['temp_air'], wind_speed=weather_data['wind_speed'], u_c=26.3, u_v=2,
            eta_m=None, module_efficiency=0.17, alpha_absorption=0.9)

    '''
    # Calculate the photocurrent, saturation current, and resistance values for the single diode model
    IL, I0, Rse, Rsh, nNsVth = pvlib.pvsystem.calcparams_desoto(
        total_irradiance['poa_global'],
        temp_cell_fpv,
        module['alpha_sc'],
        module['a_ref'],
        module['I_L_ref'],
        module['I_o_ref'],
        module['R_sh_ref'],
        module['R_s'],
        EgRef=1.121,  # This is the default value for crystalline silicon cells
    )
    #effective_irradiance, temp_cell, alpha_sc, a_ref,
    '''
    # This would replace your DeSoto model calculations
    sapm_out_fpv = pvlib.pvsystem.sapm(
        total_irradiance['poa_global'],
        temp_cell_fpv,
        module
    )

    dc = sapm_out_fpv['p_mp']
    ac = pvlib.inverter.sandia(dc, sapm_out_fpv['v_mp'], inverter)

    # Calculate the single diode model values
    #single_diode_output_fpv = pvlib.pvsystem.singlediode(IL, I0, Rse, Rsh, nNsVth)
    #dc = single_diode_output_fpv['p_mp']
    #ac = pvlib.inverter.sandia(single_diode_output_fpv['v_mp'], single_diode_output_fpv['p_mp'],inverter)

    # Extract the DC power from the single diode model output

    ## GROUND PV

    temp_cell_pv = pvsyst_cell_pv(poa_global=total_irradiance['poa_global'], temp_air=weather_data['temp_air'], wind_speed=weather_data['wind_speed'], u_c=25, u_v=1.2,
            eta_m=None, module_efficiency=0.17, alpha_absorption=0.9)
    '''
    # Calculate the photocurrent, saturation current, and resistance values for the single diode model
    IL, I0, Rse, Rsh, nNsVth = pvlib.pvsystem.calcparams_desoto(
        total_irradiance['poa_global'],
        temp_cell_pv,
        module['alpha_sc'],
        module['a_ref'],
        module['I_L_ref'],
        module['I_o_ref'],
        module['R_sh_ref'],
        module['R_s'],
        EgRef=1.121,  # This is the default value for crystalline silicon cells
    )
    '''
    sapm_out_pv = pvlib.pvsystem.sapm(
        total_irradiance['poa_global'],
        temp_cell_pv,
        module
    )

    dc_ground_pv= sapm_out_pv['p_mp']
    ac_ground_pv = pvlib.inverter.sandia(dc, sapm_out_pv['v_mp'], inverter)


    # Calculate the single diode model values
    #single_diode_output_pv = pvlib.pvsystem.singlediode(IL, I0, Rse, Rsh, nNsVth)
    #dc_ground_pv = single_diode_output_pv['p_mp']
    #ac_ground_pv = pvlib.inverter.sandia(single_diode_output_pv['v_mp'], single_diode_output_pv['p_mp'], inverter)

    ## inverter loss added
    P_nom = inverter['Paco']
    ac = np.minimum(ac, P_nom)
    ac_ground_pv = np.minimum(ac_ground_pv, P_nom)



    # Scale the PV output by the system size
    ac_scaled_fpv = (ac / (module['Vmpo']*module['Impo'])) * system_size
    ac_scaled_fpv2 = (ac / (module['Vmpo']*module['Impo'])) * system_size * (np.cos(np.deg2rad(array_tilt)) + 1.2 * np.sin(np.deg2rad(array_tilt)))

    ac_scaled_ground_pvff = (ac_ground_pv / (module['Vmpo']*module['Impo'])) * system_size
    ac_scaled_ground_pv2 = (ac_ground_pv / (module['Vmpo']*module['Impo'])) * system_size * (np.cos(np.deg2rad(array_tilt)) + 1.2 * np.sin(np.deg2rad(array_tilt)))

    cos_tilt = np.cos(np.deg2rad(array_tilt))
    sin_tilt = 1.2 * np.sin(np.deg2rad(array_tilt))
    ac_scaled_fpv2 = ac / (Array_Ms * Array_Mp * single_module_area * (cos_tilt + sin_tilt)) * 10**3
    ac_scaled_ground_pv2 = ac_ground_pv / (Array_Ms * Array_Mp * single_module_area * (cos_tilt + sin_tilt)) * 10**3


    ac_scaled_fpv3 = ac * N_inverters
    ac_scaled_ground_pv3=ac_ground_pv* N_inverters

    #ac_scaled_fpv_monthly = ac_scaled_fpv.resample('M').sum()
    #ac_scaled_fpv2_monthly = ac_scaled_fpv2.resample('M').sum()
    #ac_scaled_ground_pv_monthly = ac_scaled_ground_pv.resample('M').sum()
    #ac_scaled_ground_pv2_monthly = ac_scaled_ground_pv2.resample('M').sum()



    # Adjust the monthly energy outputs for losses
    #adjusted_monthly_fpv_energy = ac_scaled_fpv_monthly * (1 - total_pvwatts_loss_factor / 100)
    #adjusted_monthly_pv_energy = ac_scaled_ground_pv_monthly * (1 - total_pvwatts_loss_factor / 100)

    # Adjust the annual energy outputs for losses
    #adjusted_annual_fpv_energy = adjusted_monthly_fpv_energy.sum()
    #adjusted_annual_pv_energy = adjusted_monthly_pv_energy.sum()


    monthly_fpv_energy = ac_scaled_fpv3.resample('M').sum()* (1 - total_pvwatts_loss_factor / 100)  # This already contains monthly sums
    monthly_pv_energy = ac_scaled_ground_pv3.resample('M').sum()* (1 - total_pvwatts_loss_factor / 100)  # This already contains monthly sums
    annual_fpv_energy = monthly_fpv_energy.sum()/1000000
    annual_pv_energy = monthly_pv_energy.sum()/1000000
    # Create a dictionary to store all the data
    data_dict = {
        'reservoir_name': reservoir_name,
        'latitude': lat,
        'longitude': lon,
        'Annual_FPV_Energy_MWh': annual_fpv_energy,
        'Annual_PV_Energy_MWh': annual_pv_energy,
        'state': state1,
        'reservoir_area': ReservoirArea,
        'PanelArea_km2':PanelArea
    }

    # Add monthly data to the dictionary
    for i, (fpv, pv) in enumerate(zip(monthly_fpv_energy, monthly_pv_energy)):
        data_dict[f'Month_{i+1}_FPV_Energy_kWh'] = fpv/1000
        data_dict[f'Month_{i+1}_PV_Energy_kWh'] = pv/1000

    # Append this data to the energy_data DataFrame
    energy_data = energy_data.append(data_dict, ignore_index=True)





Running case number: 0 grid: 0
Debug: Reservoir name inside parse_psm3: Itaipu, Parana
Debug: Reservoir name inside download_weather_data: Itaipu, Parana


<ipython-input-16-04f695e22c0b>:675: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  energy_data = energy_data.append(data_dict, ignore_index=True)


Running case number: 1 grid: 1
Debug: Reservoir name inside parse_psm3: Passo rea, Rio Grande do Sul
Debug: Reservoir name inside download_weather_data: Passo rea, Rio Grande do Sul


<ipython-input-16-04f695e22c0b>:675: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  energy_data = energy_data.append(data_dict, ignore_index=True)


Running case number: 2 grid: 2
Debug: Reservoir name inside parse_psm3: Campos novos, Santa Catarina
Debug: Reservoir name inside download_weather_data: Campos novos, Santa Catarina


...

<ipython-input-16-04f695e22c0b>:675: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  energy_data = energy_data.append(data_dict, ignore_index=True)


Running case number: 4 grid: 4
Debug: Reservoir name inside parse_psm3: Ludesa, Santa Catarina
Debug: Reservoir name inside download_weather_data: Ludesa, Santa Catarina


<ipython-input-16-04f695e22c0b>:675: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  energy_data = energy_data.append(data_dict, ignore_index=True)


Running case number: 5 grid: 5
Debug: Reservoir name inside parse_psm3: Arroio duro, Rio Grande do Sul
Debug: Reservoir name inside download_weather_data: Arroio duro, Rio Grande do Sul


<ipython-input-16-04f695e22c0b>:675: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  energy_data = energy_data.append(data_dict, ignore_index=True)


In [ ]:
# Save the energy_data DataFrame to a CSV file
energy_data.to_csv('energy_data.csv', index=False)
